# Build pipeline with command function

**Requirements** - In order to benefit from this tutorial, you will need:
- A basic understanding of Machine Learning
- An Azure account with an active subscription. [Create an account for free](https://azure.microsoft.com/free/?WT.mc_id=A261C142F)
- An Azure ML workspace. [Check this notebook for creating a workspace](/sdk/resources/workspace/workspace.ipynb) 
- A Compute Cluster. [Check this notebook to create a compute cluster](/sdk/resources/compute/compute.ipynb)
- A python environment
- Installed Azure Machine Learning Python SDK v2 - [install instructions](/sdk/README.md#getting-started)

**Learning Objectives** - By the end of this tutorial, you should be able to:
- Connect to your AML workspace from the Python SDK
- Define `commandComponent` using command() function
- Create `pipeline` using component defined by command() function

**Motivations** - This notebook explains how to define `commandComponent` via command function, then use command component to build pipeline. The command component is a fundamental construct of Azure Machine Learning pipeline. It can be used to run a task on a specified compute (either local or on the cloud). The command component accepts `environment` to setup required infrastructure. You can define a `command` to run on this infrastructure with `inputs`. You can reuse the same `component` with different pipeline.  

# 1. Connect to Azure Machine Learning Workspace

The [workspace](https://docs.microsoft.com/en-us/azure/machine-learning/concept-workspace) is the top-level resource for Azure Machine Learning, providing a centralized place to work with all the artifacts you create when you use Azure Machine Learning. In this section we will connect to the workspace in which the job will be run.

## 1.1. Import the required libraries

In [ ]:
#import required libraries
from azure.identity import InteractiveBrowserCredential
from azure.ml import MLClient, dsl, command
from azure.ml.entities import JobInput, JobOutput

## 1.2. Configure workspace details and get a handle to the workspace

To connect to a workspace, we need identifier parameters - a subscription, resource group and workspace name. We will use these details in the `MLClient` from `azure.ml` to get a handle to the required Azure Machine Learning workspace. We use the default [interactive authentication](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity.interactivebrowsercredential?view=azure-python) for this tutorial. More advanced connection methods can be found [here](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity?view=azure-python).

In [ ]:
#Enter details of your AML workspace
subscription_id = '<SUBSCRIPTION_ID>'
resource_group = '<RESOURCE_GROUP>'
workspace = '<AML_WORKSPACE_NAME>'

In [ ]:
#get a handle to the workspace
ml_client = MLClient(InteractiveBrowserCredential(), subscription_id, resource_group, workspace)

# 2. Define Command object via command function 
Use `command` function to create a `Command` object which can be used in `dsl.pipeline`


In [ ]:
download_url = "https://dprepdata.blob.core.windows.net/demo/Titanic.csv"
file_name = "data.csv"
column_name = "Cabin"
environment = "AzureML-sklearn-0.24-ubuntu18.04-py37-cpu:1"
# 1. Create a command component to download a input data
download_data = command(
    name="download-input",
    display_name="Download Data",
    description="Download a input from remote URL and return it in output.",
    tags=dict(),
    command="curl ${{inputs.url}} > ${{outputs.output_folder}}/${{inputs.file_name}}",
    environment=environment,
    inputs=dict(
        url=download_url,
        file_name=file_name
    ),
    outputs=dict(
        output_folder=JobOutput(type="uri_folder", mode="rw_mount")
    )
)
# 2. Create a Python script command to filter data
filter_data = command(
    name="filter-data",
    display_name="Filter Data",
    description="Filter some columns.",
    tags=dict(),
    command="python parse_input.py --input ${{inputs.input_folder}}/${{inputs.file_name}} --output ${{outputs.output_folder}}/${{inputs.file_name}} --column_name ${{inputs.column_name}}",
    environment=environment,
    code="./parse_input.py",
    inputs=dict(
        input_folder=JobInput(type="uri_folder"),
        file_name=file_name,
        column_name=column_name
    ),
    outputs=dict(
        output_folder=JobOutput(type="uri_folder")
    )
)
# 3. Create a command component to show inputs
show_data = command(
    name="show-data",
    display_name="Show Data",
    description="Show data in command line and return it in output.",
    tags=dict(),
    command="cat ${{inputs.input_folder}}/${{inputs.file_name}} && cp ${{inputs.input_folder}}/${{inputs.file_name}} ${{outputs.output_folder}}/${{inputs.file_name}}",
    environment=environment,
    inputs=dict(
        input_folder=JobInput(type="uri_folder"),
        file_name=file_name
    ),
    outputs=dict(
        output_folder=JobOutput(type="uri_folder")
    )
)

# 3. Basic pipeline job

## 3.1 Build pipeline

Build a pipeline with predefined commands.

In [ ]:
compute = 'cpu-cluster'
@dsl.pipeline(
    description="The hello world pipeline job",
    tags={"owner": "sdkteam", "tag": "tagvalue"},
    compute=compute,
    experiment_name="pipeline-with-command-function"
)
def pipeline(url, file_name):
    download_data_node = download_data(url=url, file_name=file_name)
    download_data_node.compute = compute
    filter_data_node = filter_data(input_folder=download_data_node.outputs.output_folder, file_name=file_name)
    filter_data_node.compute = compute
    show_data_node = show_data(input_folder=filter_data_node.outputs.output_folder, file_name=file_name)
    show_data_node.compute = compute

    return {
        "output_file": show_data_node.outputs.output_folder
    }

pipeline = pipeline(download_url, file_name)

# 3.2 Submit pipeline job

In [ ]:

pipeline_job = ml_client.create_or_update(pipeline)
pipeline_job.services["Studio"].endpoint

# Next Steps
You can see further examples of running a pipeline job [here](/sdk/jobs/pipelines/)